In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time

import numpy as np
import matplotlib.pyplot as plt

import sys, os

import moyra as ma
import moyra.elements as ele
import moyra.forces as ef

me.mechanics_printing()

#### Create Symbols

In [2]:
DoFs = 2          # total degrees of freedom
main_panels = 10  # main wing panels
htp_panels = 4    # horizontal tailplane panels

p = ma.ModelParameters.DynamicModel(DoFs)
p.L_ot = ma.ModelSymbol(string = 'L_ot')
p.L_tc = ma.ModelSymbol(string = 'L_tc')
p.L_tw = ma.ModelSymbol(string = 'L_tw')
p.L_te = ma.ModelSymbol(string = 'L_te')

p.m = ma.ModelSymbol(string = 'm')
p.I_xx = ma.ModelSymbol(string = 'I_xx')
p.I_yy = ma.ModelSymbol(string = 'I_yy')
p.I_zz = ma.ModelSymbol(string = 'I_zz')

p.y_i = sym.Symbol('y_i') # spanwise location

## Numeric Model Constants
p.a = ma.ModelMatrix(value =[np.pi*2]*main_panels,symbols=sym.symbols(f'a_m:{main_panels}')) # local 2d lift curve slope at each panel on main wing
p.a_h = ma.ModelMatrix(value =[np.pi*2]*htp_panels,symbols=sym.symbols(f'a_htp:{htp_panels}')) # local 2d lift curve slope at each panel on horizontal tailplane
p.a_0 = ma.ModelSymbol(value = 2*np.pi, string = 'a_0')   # placeholder for local c_l


p.c = ma.ModelSymbol(value = 0.15,string = 'c') # chord of wing
p.c_h = ma.ModelSymbol(value = 0.15,string = 'c_htp') # chord of htp

p.b = ma.ModelSymbol(value = 0.15,string = 'b') # chord of wing
p.b_h = ma.ModelSymbol(value = 0.15,string = 'b_htp') # chord of htp

# Attitude Parmas
p.rho = ma.ModelSymbol(value = 1.225,string = 'rho')                 # density
p.V = ma.ModelSymbol(value = 10,string = 'V')                        # velocity
p.g  = ma.ModelSymbol(value = 9.81,string = 'g')                     # gravity


#### Create Coordinate Systems

In [3]:
tether_frame = ma.HomogenousTransform().R_z(p.q[0]).R_x(p.q[1]).Translate(0,0,-p.L_ot)

#Generate Mass Matrices
M = ele.MassMatrix(p.m, I_xx = p.I_xx, I_yy=p.I_yy, I_zz=p.I_zz)
M = ele.MassMatrix(p.m, I_xx = 0, I_yy=0, I_zz=0)


# Generate Rigid Elements
aircraft_mass = ele.RigidElement(tether_frame.Translate(p.L_tc,0,0),M,gravityPotential=True)

# Main Wing Aero Forces 
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,tether_frame.Translate(p.L_tw,p.y_i,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = 0,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True,
                                z_inverted=False)

# HTP Aero Forces 
htp_AeroForces = ef.AeroForce.PerUnitSpan(p,tether_frame.Translate(p.L_te,p.y_i,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = 0,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c_h,
                               c_d_max = 0,
                               linear = True,
                                z_inverted=False)


In [4]:
# split wing into segments
forces = []
for i in range(main_panels):
    seg_width = p.b/main_panels
    yi = seg_width/2 + i*seg_width - p.b/2
    forces.append(wing_AeroForces.subs({p.y_i:yi,p.a_0:p.a[main_panels-(i+1)]})*seg_width)
for i in range(htp_panels):
    seg_width = p.b_h/htp_panels
    yi = seg_width/2 + i*seg_width - p.b_h/2
    forces.append(wing_AeroForces.subs({p.y_i:yi,p.a_0:p.a_h[htp_panels-(i+1)]})*seg_width)
Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
AeroForces = ef.ExternalForce(Q)

In [5]:
## print the velocity in the wing frame
tether_frame = ma.HomogenousTransform().R_z(p.q[0]).R_x(p.q[1]).Translate(0,0,-p.L_ot)
T = tether_frame.Translate(p.L_tw,p.y_i,0)
BJ = sym.trigsimp(sym.powsimp(sym.cancel(sym.expand(T.BodyJacobian(p.q)))))
sym.simplify(BJ*p.qd)

/Users/fintan/Git/FwtModels/venv/lib/python3.7/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_png function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/Users/fintan/Git/FwtModels/venv/lib/python3.7/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_rgba function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/Users/fintan/Git/FwtModels/venv/lib/python3.7/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_mask function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/Users/fintan/Git/FwtModels/venv/lib/python3.7/site-packages/IPyt

⎡-(Lₒₜ⋅sin(q₁) + yᵢ⋅cos(q₁))⋅q₀̇⎤
⎢                              ⎥
⎢   Lₒₜ⋅q₁̇ + L_tw⋅cos(q₁)⋅q₀̇   ⎥
⎢                              ⎥
⎢   -L_tw⋅sin(q₁)⋅q₀̇ + yᵢ⋅q₁̇   ⎥
⎢                              ⎥
⎢              q₁̇              ⎥
⎢                              ⎥
⎢          sin(q₁)⋅q₀̇          ⎥
⎢                              ⎥
⎣          cos(q₁)⋅q₀̇          ⎦

In [6]:
sm = ma.SymbolicModel.FromElementsAndForces(p,[aircraft_mass],ExtForces=AeroForces)

In [7]:
sm.M

⎡   2      2           2                      ⎤
⎢Lₒₜ ⋅m⋅sin (q₁) + L_tc ⋅m  Lₒₜ⋅L_tc⋅m⋅cos(q₁)⎥
⎢                                             ⎥
⎢                                    2        ⎥
⎣   Lₒₜ⋅L_tc⋅m⋅cos(q₁)            Lₒₜ ⋅m      ⎦

In [8]:
sym.simplify(sm.f)

⎡Lₒₜ⋅m⋅(2⋅Lₒₜ⋅cos(q₁)⋅q₀̇ - L_tc⋅q₁̇)⋅sin(q₁)⋅q₁̇⎤
⎢                                             ⎥
⎢          ⎛                2    ⎞            ⎥
⎣    Lₒₜ⋅m⋅⎝- Lₒₜ⋅cos(q₁)⋅q₀̇  + g⎠⋅sin(q₁)    ⎦

In [9]:
sm.ExtForces.Q()

⎡                 ⎛               9⋅b⋅cos(q₁)⎞ ⎛                   9⋅b⋅q₁̇⎞   
⎢  L_tw⋅b⋅c⋅ρ⋅aₘ₀⋅⎜-Lₒₜ⋅sin(q₁) - ───────────⎟⋅⎜-L_tw⋅sin(q₁)⋅q₀̇ + ──────⎟⋅si
⎢                 ⎝                    20    ⎠ ⎝                     20  ⎠    
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                          20                                 
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣                                                                             

                          ⎛               7⋅b⋅cos(q₁)⎞ ⎛                   7⋅b
n(q₁)⋅q₀̇   L_tw⋅b⋅c⋅ρ⋅aₘ₁⋅⎜-Lₒₜ⋅sin(q₁) - ────────

sm.t